# 📘 Hackathon Project Video Evaluator using PraisonAIAgents

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/Hackathon_Agent_Notebook.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents opencv-python moviepy

# Set OpenAI API Key for Authentication

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Enter your api key"
print("OpenAI API key has been set successfully!")

# Setup: Imports and Agent Frameworks

In [ ]:
import os
import json
import re
import cv2
import numpy as np
from typing import List
from pydantic import BaseModel
from praisonaiagents import Agent, Task, PraisonAIAgents

## 🛠️ Tools and Data Models

In [ ]:
class ProjectEvaluation(BaseModel):
    innovation_score: int
    technical_complexity: int
    presentation_quality: int
    user_experience: int
    completeness: int
    overall_score: int
    key_strengths: List[str]
    areas_for_improvement: List[str]
    notable_features: List[str]
    technical_highlights: List[str]
    recommendations: List[str]
    market_potential: str
    scalability_assessment: str

## 🎞️ Video Frame Extraction

In [ ]:
def extract_frames(video_path: str, num_frames: int = 5) -> List[str]:
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video not found: {video_path}")
    frames_dir = os.path.join(os.path.dirname(video_path), "frames")
    os.makedirs(frames_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // (num_frames + 1)
    paths = []
    for i in range(1, num_frames + 1):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret: break
        path = os.path.join(frames_dir, f"frame_{i}.jpg")
        cv2.imwrite(path, frame)
        paths.append(path)
    cap.release()
    return paths

## 🤖 Define the AI Agent

In [ ]:
hackathon_judge = Agent(
    name="HackathonJudge",
    role="Technical Project Evaluator",
    goal="Evaluate hackathon projects through video demonstrations",
    backstory="""You are an expert hackathon judge. You give fair scores and detailed feedback
    on technical complexity, innovation, presentation, and UX.""",
    llm="gpt-4o-mini",
    self_reflect=False
)

## 🚀 Main Evaluation Function

In [ ]:
def evaluate_project(video_path: str) -> ProjectEvaluation:
    frame_paths = extract_frames(video_path)
    task = Task(
        name="project_evaluation",
        description="""Analyze hackathon video frames and score: innovation, complexity, UX,
        presentation, completeness. Provide strengths, improvements, highlights, market potential,
        and recommendations.""",
        expected_output="JSON feedback with scores and suggestions",
        agent=hackathon_judge,
        output_pydantic=ProjectEvaluation,
        images=frame_paths
    )
    agents = PraisonAIAgents(
        agents=[hackathon_judge],
        tasks=[task],
        process="sequential",
        verbose=True
    )

    try:
        response = agents.start()
        if isinstance(response, dict) and 'task_results' in response:
            output = response['task_results'][0]
            if hasattr(output, 'pydantic'):
                return output.pydantic
            elif hasattr(output, 'raw'):
                raw = output.raw.strip()
                json_text = re.search(r'```(?:json)?\s*([\s\S]+?)\s*```', raw)
                raw = json_text.group(1).strip() if json_text else raw
                return ProjectEvaluation(**json.loads(raw))
        elif isinstance(response, str):
            return ProjectEvaluation(**json.loads(response))
        return ProjectEvaluation(**response)
    finally:
        for path in frame_paths:
            os.remove(path)
        try:
            os.rmdir(os.path.join(os.path.dirname(video_path), "frames"))
        except: pass

## ▶️ Run the Evaluation

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-...YOUR_API_KEY..."

video_file = "presentation.mp4"
try:
    result = evaluate_project(video_file)
    print("\n🔍 Hackathon Project Evaluation")
    print("=" * 40)
    print(f"Overall Score: {result.overall_score}/100")
    print(f"Innovation: {result.innovation_score}/100")
    print(f"Technical Complexity: {result.technical_complexity}/100")
    print(f"Presentation Quality: {result.presentation_quality}/100")
    print(f"User Experience: {result.user_experience}/100")
    print(f"Completeness: {result.completeness}/100")
    print("\n✅ Key Strengths:")
    for s in result.key_strengths: print(f"- {s}")
    print("\n⚠️ Areas for Improvement:")
    for a in result.areas_for_improvement: print(f"- {a}")
    print("\n🛠️ Technical Highlights:")
    for t in result.technical_highlights: print(f"- {t}")
    print("\n📌 Recommendations:")
    for r in result.recommendations: print(f"- {r}")
    print(f"\n📈 Market Potential: {result.market_potential}")
    print(f"\n📊 Scalability Assessment: {result.scalability_assessment}")
except FileNotFoundError:
    print("❗ 'presentation.mp4' not found. Please add it to the current directory.")
except Exception as e:
    print(f"❌ Evaluation Error: {e}")

In [2]:
output = """
╭─ Agent Info ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                              │
│  👤 Agent: HackathonJudge                                                                                                    │
│  Role: Technical Project Evaluator                                                                                           │
│                                                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Generating... 6.6s ─────────────────────────────────────────────────────╮
│                                                                                                                              │
│  {                                                                                                                           │
│    "innovation_score": 30,                                                                                                   │
│    "technical_complexity": 20,                                                                                               │
│    "presentation_quality": 25,                                                                                               │
│    "user_experience": 35,                                                                                                    │
│    "completeness": 20,                                                                                                       │
│    "overall_score": 24,                                                                                                      │
│    "key_strengths": [                                                                                                        │
│      "Simple concept",                                                                                                       │
│      "Clear visual representation"                                                                                           │
│    ],                                                                                                                        │
│    "areas_for_improvement": [                                                                                                │
│      "Enhance technical features",                                                                                           │
│      "Improve presentation clarity",                                                                                         │
│      "Add user interaction elements"                                                                                         │
│    ],                                                                                                                        │
│    "notable_features": [                                                                                                     │
│      "Basic visual interface"                                                                                                │
│    ],                                                                                                                        │
│    "technical_highlights": [                                                                                                 │
│      "Use of basic graphics"                                                                                                 │
│    ],                                                                                                                        │
│    "recommendations": [                                                                                                      │
│      "Incorporate user feedback mechanisms",                                                                                 │
│      "Develop a more robust backend",                                                                                        │
│      "Enhance visual aesthetics"                                                                                             │
│    ],                                                                                                                        │
│    "market_potential": "Limited due to simplicity",                                                                          │
│    "scalability_assessment": "Low, requires significant development"                                                         │
│  }                                                                                                                           │
│                                                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
"""
print(output)



╭─ Agent Info ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                              │
│  👤 Agent: HackathonJudge                                                                                                    │
│  Role: Technical Project Evaluator                                                                                           │
│                                                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Generating... 6.6s ─────────────────────────────────────────────────────╮
│                                                                                               